In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage

In [2]:
def gamma_correction(img, gamma):
    gamma_cvt = np.array([[255 * (float(i)/255) ** (1.0/gamma)] for i in range(256)], dtype = 'uint8')
    return cv2.LUT(img, gamma_cvt)

In [3]:
def contrast_correction(img, c):
    contrast_cvt = np.array([255.0 / (1 + np.exp(-c * (i - 128) / 255)) for i in np.arange(0, 256)], dtype ="uint8")
    return cv2.LUT(img, contrast_cvt)

In [4]:
def edge_detection(img, ksize):
    img = np.array(img,dtype='int')-np.array(cv2.GaussianBlur(img, (ksize,ksize), 0),dtype='int')
    return np.array(np.clip(img,0,255),dtype='uint8')

In [30]:
def clahe(img, tilesize):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(tilesize,tilesize))
    v = clahe.apply(v)
    hsv = cv2.merge((h,s,v))
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [5]:
def edge_emphasize(img, ksize, k):
    img = (1+k)*np.array(img,dtype='int')-k*np.array(cv2.GaussianBlur(img, (ksize,ksize), 0),dtype='int')
    return np.array(np.clip(img, 0, 255),dtype='uint8')

In [31]:
def bilateral(img):
    return cv2.bilateralFilter(frame, 15, 20, 20)

In [32]:
def nlmf(img):
    return cv2.fastNlMeansDenoisingColored(frame,None,10,10,7,21)

In [33]:
def filter2D(img):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]], np.float32)
    return cv2.filter2D(frame, -1, kernel)

In [36]:
in_video = cv2.VideoCapture("video.mov")

width = int(in_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(in_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
frame_count = int(in_video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(in_video.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out_video = cv2.VideoWriter("output.mov", fourcc, frame_rate, size)

while in_video.isOpened():
    ret, frame = in_video.read()
    # 最後まで読み切ったら終了
    if not ret:
        break
    
    frame = gamma_correction(frame, 2.0)
    
    frame = clahe(frame,7)
    
    frame = filter2D(frame)
    
    frame = edge_emphasize(frame, 5, 5)
    
    frame = bilateral(frame)
    
    frame = nlmf(frame)

    out_video.write(frame)
    
in_video.release()
out_video.release()
cv2.destroyAllWindows() 
print("end")


end
